## 作者的话

>一开始呢，是看了秋叶的阿里云视频，然后在阿里云进行lora训练，结果总是环境出错，而在当时也没有这样的训练教程出现。

>于是我在多方指导的基础上，用低级镜像完成了简化版lora的制作也就是第一个视频，但是效果呢不尽如人意，但是呢却意外的有着不错的播放量，为了不辜负大家的期待，我开始从镜像学起重新制作，但是这个时候，我突然发现了一个问题。

>那就是我没有可以和a10或v100与之相配的显卡，那么我要如何制作镜像呢，而这个时候我开始转变思路租阿里云的免费服务器，但是服务器连显卡都没有根本装不上驱动

>后来我尝试在机器学习里进行docker的镜像训练，但经过调查运行机器学习的本身便是docker。镜像我是配不出来了，所以我开始利用秋叶的镜像，进行降级测试，解决了出现的大部分问题。完成了上一个最终版，历时近一个月。

>而现在呢我根据秋叶最新发布的lora训练一键界面，修改了href之后配置在了阿里云上，进一步提高了界面友好度，感谢赛博佛祖。

>在这里感谢支持我的粉丝，和给予我帮助的大佬们，如果大家喜欢还请给我点点关注一键三连。

>6月22日最新版

# **基于Bilibili UP主：[秋葉aaaki]发布的[保姆式LoRA模型一键包文件](https://www.bilibili.com/video/BV1fs4y1x7p2/)**
最核心的文件的整合代码均由主要作者[秋葉aaaki](https://github.com/Akegarasu/lora-scripts)完成。
开始前**建议阅读**：
1.   [保姆式LoRA模型一键包文件](https://www.bilibili.com/video/BV1AL411q7Ub/?spm_id_from=333.337.search-card.all.click&vd_source=e6294619bd5137ea0025a3d001e715a7)
2.   [参数心得](https://www.bilibili.com/video/BV1GM411E7vk/)
3.   [训练教程](https://www.bilibili.com/video/BV1fs4y1x7p2/?spm_id_from=333.999.0.0)
4.   [阿里云部署教程](https://www.bilibili.com/video/BV1fs4y1x7p2/?spm_id_from=333.999.0.0)
> 本人只是完成阿里云下的依赖安装，使用的下载源不保证长期有效。本人对python与linux的使用能力有限，代码在各位大佬的指导和WSH032大佬于colab的依赖代码修改下完成，不足部分，有兴趣者可修改并分享。

> *--分享的责任与获取的自由*
**待解决问题**:

> 1.关于数据集的挂载免费领用的教程，我这边没有相应教程如有需要可自行解决

> 2.总是有人说觉得阿里云的环境有问题比本地更容易欠拟或者过拟，不知道是少了哪一步。

更新内容：
已经完成了秋叶于阿里云的web部署，金字塔噪声，分层训练，梯度都可以使用了

采用了群友指月的修复代码，现已解决了张量版

根据其它群友的意见对代码进行了升级

训练包版本为2023年6月22日版

已突破抱脸封锁，所有文件均不使用抱脸下载，不会再出现下载失败的情况了，

## 一、基础依赖和训练包拉取(时间会更长)

In [ ]:
#查看是什么GPU
!nvidia-smi
#安装解压软件和下载存储工具
!apt-get update  > /dev/null 2>&1 
!apt-get install zip -y > /dev/null 2>&1
!apt install -y aria2 > /dev/null 2>&1 
#进入根目录
%cd /mnt/workspace/
#删除先前下载的lora训练文件
!mkdir -p /mnt/workspace/lora-scripts/  #防止报错
!rm -r /mnt/workspace/lora-scripts/
#下载训练文件
!cd /mnt/workspace/ && wget -c https://liblibai-online.oss-cn-beijing.aliyuncs.com/web/model/e0921b4a5f3f49fc49b36860d38345c47cb24395161b1aaef71e94c4e7040637.ckpt  -O 1.zip
!unzip /mnt/workspace/1.zip -d /mnt/workspace/ > /dev/null 2>&1 && echo 解压成功
!sudo rm -r /mnt/workspace/1.zip  > /dev/null 2>&1  && echo 删除压缩包成功
#安装依赖
!pip -q install torch==2.0.0 torchvision xformers triton    
#进入目录
%cd /mnt/workspace/lora-scripts/sd-scripts/
#安装目录要求依赖
!pip -q  install --upgrade -r requirements.txt && echo 安装需求依赖成功
#安装训练指示器
!pip -q install --upgrade lion-pytorch lycoris-lora dadaptation  && echo 安装训练器成功
#进行bitsandbytes的重装
!pip uninstall -q bitsandbytes -y > /dev/null 2>&1 && echo 卸载成功
!pip install  -q --no-cache-dir bitsandbytes[full] > /dev/null 2>&1    && echo 安装成功

## 二、训练集下载（可自行改变路径）

In [ ]:
%cd /mnt/workspace/lora-scripts/ 
!mkdir -p /mnt/workspace/lora-scripts/train/  #防止首次运行报错生成文件夹
!rm -r /mnt/workspace/lora-scripts/train/     #删除测试文件夹
!mkdir -p /mnt/workspace/lora-scripts/train/aki #建立下载目录

### 只修改下方链接即可替换数据集

In [ ]:
!cd /mnt/workspace/lora-scripts/train/aki/  && wget -c https://liblibai-online.liblibai.com/web/model/55b62e7b5a3e4c7bff58ae30b32f2f5f31b750d40872302d729230d0904f9c33.safetensors -O 1.zip > /dev/null 2>&1 && echo 下载成功 #下载训练集

In [ ]:
!unzip /mnt/workspace/lora-scripts/train/aki/1.zip -d /mnt/workspace/lora-scripts/train/aki/ > /dev/null 2>&1 && echo 解压成功   #解压训练集压缩包
!sudo rm -r /mnt/workspace/lora-scripts/train/aki/1.zip  > /dev/null 2>&1  && echo 删除压缩包成功 #删除训练集压缩包

## 三、模型下载（可自行改变路径）

### 只修改下方链接即可替换底膜

In [ ]:
!cd /mnt/workspace/lora-scripts/sd-models/ && wget -c  https://liblibai-online.liblibai.com/models/e7e55f800035004cf0ee31d91f874a415a2b64a4.safetensors -O down.safetensors  #下载底部模型

默认底膜：Chilloutmix-Ni-pruned-fp32-fix
https://liblibai-online.liblibai.com/models/e7e55f800035004cf0ee31d91f874a415a2b64a4.safetensors

其它底膜链接：

majicMIX realistic
https://liblibai-online.liblibai.com/web/model/e4a30e4607faeb06b5d590b2ed8e092690c631da0b2becb6224d4bb5327104b7.safetensors


## 四、启动秋葉aaaki的一键式lora训练界面

In [ ]:
%cd /mnt/workspace/lora-scripts    
#跳转到UI文件
!bash run_gui.sh #界面文件点击127.0。0.0.1即可进入ui

# **基于Bilibili UP主：[秋葉aaaki]的阿里云webui脚本修改而来**

## 一、webui主体安装

In [ ]:
%cd /mnt/workspace/
!git clone https://gitcode.net/overbill1683/stable-diffusion-webui
%cd stable-diffusion-webui
!mkdir repositories
%cd repositories
!git clone "https://gitcode.net/overbill1683/stablediffusion" "stable-diffusion-stability-ai"
!git clone "https://gitcode.net/overbill1683/taming-transformers" "taming-transformers"
!git clone "https://gitcode.net/overbill1683/k-diffusion" "k-diffusion"
!git clone "https://gitcode.net/overbill1683/CodeFormer" "CodeFormer"
!git clone "https://gitcode.net/overbill1683/BLIP" "BLIP"
%cd /mnt/workspace/stable-diffusion-webui
!wget -O "config.json" "https://gitcode.net/Akegarasu/sd-webui-configs/-/raw/master/config.json"

## 二、webui插件安装（可自行加入或替换插件）

In [ ]:
## 安装常用插件
extensions = [
    "https://gitcode.net/ranting8323/a1111-sd-webui-tagcomplete",
    "https://gitcode.net/ranting8323/stable-diffusion-webui-localization-zh_CN",
    "https://gitcode.net/ranting8323/sd-webui-additional-networks",
]

%cd /mnt/workspace/stable-diffusion-webui
for e in extensions:
    !git -C "extensions" clone {e}

##   三、底膜和lora移动用于测试（如不想移动可跳过但要自行下载底膜）

###  底膜移动

In [ ]:
#移动底膜到指定文件夹
!mv /mnt/workspace/lora-scripts/sd-models/*   /mnt/workspace/stable-diffusion-webui/models/Stable-diffusion/

###  lora移动

In [ ]:
!mv /mnt/workspace/lora-scripts/output/*   /mnt/workspace/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora

## 四、启动webUI

In [ ]:
import os
package_envs  = [
      {
        "env": "GFPGAN_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/GFPGAN.git@8d2447a2d918f8eba5a4a01463fd48e45126a379"
      },
      {
        "env": "CLIP_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/CLIP.git@d50d76daa670286dd6cacf3bcd80b5e4823fc8e1"
      },
      {
        "env": "OPENCLIP_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/open_clip.git@bb6e834e9c70d9c27d0dc3ecedeebeaeb1ffad6b"
      }
]
os.environ["PIP_INDEX_URL"] = "https://mirrors.aliyun.com/pypi/simple/"
for i in package_envs:
    os.environ[i["env"]] = i["url"]
%cd /mnt/workspace/stable-diffusion-webui
!python launch.py --no-download-sd-model --xformers --share --listen

# 可选代码

## 一、换一个阿里云文件管理ui的方法

In [ ]:
# 安装imjoy-elfinder（web文件浏览器）依赖
!sudo apt-get update > /dev/null 2>&1 
!sudo apt-get install -y python3-pip  > /dev/null 2>&1 
!sudo apt-get install -y libfuse-dev  > /dev/null 2>&1 
!pip3 install imjoy-elfinder         > /dev/null 2>&1 
# 启动imjoy-elfinder服务
!imjoy-elfinder --root-dir=/mnt/workspace/ --port=8076  

## 二、压缩文件代码用于方便进行下载该代码会将路径下的模型打包至/mnt/workspace/（即可见最表层目录）

In [ ]:
%cd /mnt/workspace/
!tar -zcvf 1.tar.gz /mnt/workspace/lora-scripts/output > /dev/null 2>&1  && echo 压缩训练的lora模型成功

## 三、VAE的下载

In [ ]:
## vae的下载
!rm -r /mnt/workspace/lora-scripts/vae/
!mkdir -p /mnt/workspace/lora-scripts/vae/
!cd /mnt/workspace/lora-scripts/sd-models/ && wget -c  https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt

## 四、aria2下载工具(需执行过第一模块的第一板块或单独执行下方两个代码）

In [ ]:
!apt-get update  > /dev/null 2>&1 
!apt install -y aria2 > /dev/null 2>&1 

### 私库下载方式model_url是下载链接位置可自行替换http_user为抱脸用户名http_passwd为抱脸登录密码（做了伪装代理下载成功率提高了）

In [ ]:
def aria2(url, filename, d, http_user="", http_passwd="", user_agent=""):
    command = [
        "aria2c", "--console-log-level=error", "-c", "-x 16", "-s 16", url, "-d", d, "-o", filename
    ] + (["--http-user=" + http_user] if http_user else []) + (["--http-passwd=" + http_passwd] if http_passwd else []) + (["--user-agent='" + user_agent + "'"] if user_agent else [])
    !{" ".join(command)}
# 下面三个替换是你要做的其他的都可以不动

model_url = ""  # 替换为你要下载的链接

http_user = ""    # 请将用户名替换为实际的用户名，用户名为你的抱脸用户名

http_passwd = ""  # 请将密码替换为实际的密码，密码是你的抱脸密码

filename = model_url.split("/")[-1]  # 提取文件名部分
d = "/mnt/workspace/lora-scripts/sd-models/"    # 保存文件地址默认为"/mnt/workspace/lora-scripts/sd-models/
user_agent = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"  # 别动如果你不懂

aria2(model_url, filename, d, http_user, http_passwd, user_agent)

### 公库下载方式model_url是底膜下载链接位置可自行替换（做了伪装代理下载成功率提高了）

In [ ]:
def aria2(url, filename, d, http_user="", http_passwd="", user_agent=""):
    command = [
        "aria2c", "--console-log-level=error", "-c", "-x 16", "-s 16", url, "-d", d, "-o", filename
    ] + (["--user-agent='" + user_agent + "'"] if user_agent else [])
    !{" ".join(command)}

model_url = ""  # 替换为你要下载的链接
filename = model_url.split("/")[-1]  # 提取文件名部分
d = "/mnt/workspace/lora-scripts/sd-models/"    # 保存文件地址默认为"/mnt/workspace/lora-scripts/sd-models/
user_agent = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"  # 别动如果你不懂

aria2(model_url, filename, d, user_agent=user_agent)